In [1]:
#!pip install pandas
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
#grab data
data_path = 'data21.csv'
data = pd.read_csv(data_path)
#print
data.describe()

,m1 h,m2 h,m3 h,m4 h,m1 a,m2 a,m3 a,m4 a,winner
count,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000,965.000000
mean,0.305223,-0.012321,-0.263052,-0.511524,0.246197,0.011938,-0.244872,-0.274040,0.539896
std,1.554685,0.848211,1.206167,11.889701,2.168745,0.843349,1.261578,6.352298,0.498664
min,-15.990000,-1.900000,-8.627143,-355.150000,-50.270000,-1.860000,-7.450260,-176.270000,0.000000
25%,-0.370000,-0.580000,-0.816888,-0.831284,-0.330000,-0.540000,-0.845377,-0.851875,0.000000
50%,0.430000,-0.130000,-0.073568,0.207281,0.370000,-0.090000,-0.066644,0.264211,1.000000
75%,1.160000,0.690000,0.431415,1.133483,1.130000,0.670000,0.455714,1.134621,1.000000
max,4.130000,1.820000,4.237515,5.600000,5.540000,1.920000,6.093750,5.540000,1.000000


In [3]:
#y is the value we are trying to predict
y = data.winner
#these are the variables we are using to make the prediction
features = ['m1 h', 'm2 h', 'm3 h', 'm4 h', 'm1 a', 'm2 a', 'm3 a', 'm4 a']
x = data[features]

corr = x.corr()
corr.style.background_gradient(cmap='copper').format(precision=2)

,m1 h,m2 h,m3 h,m4 h,m1 a,m2 a,m3 a,m4 a
m1 h,1.00,0.04,-0.15,0.12,0.02,0.02,-0.03,0.01
m2 h,0.04,1.00,0.01,0.05,0.01,-0.11,0.00,-0.01
m3 h,-0.15,0.01,1.00,0.07,-0.02,-0.02,-0.03,-0.03
m4 h,0.12,0.05,0.07,1.00,-0.00,-0.01,0.00,-0.00
m1 a,0.02,0.01,-0.02,-0.00,1.00,-0.02,-0.15,0.78
m2 a,0.02,-0.11,-0.02,-0.01,-0.02,1.00,-0.00,-0.01
m3 a,-0.03,0.00,-0.03,0.00,-0.15,-0.00,1.00,0.14
m4 a,0.01,-0.01,-0.03,-0.00,0.78,-0.01,0.14,1.00


In [16]:
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state = 1)

data_model = DecisionTreeRegressor(random_state=0)
data_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = data_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

0.4834710743801653


In [17]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %f" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  0.480611
Max leaf nodes: 50  		 Mean Absolute Error:  0.469145
Max leaf nodes: 500  		 Mean Absolute Error:  0.504132
Max leaf nodes: 5000  		 Mean Absolute Error:  0.504132


In [18]:
random_model = RandomForestRegressor(random_state=0)
random_model.fit(train_X, train_y)

forest_preds = random_model.predict(val_X)
print(mean_absolute_error(val_y, forest_preds))

0.4857024793388429


In [19]:
from sklearn.ensemble import GradientBoostingClassifier
class_model = GradientBoostingClassifier(random_state=1)
class_model.fit(train_X, train_y)

class_preds = class_model.predict(val_X)
print(accuracy_score(val_y, class_preds))

0.5495867768595041


In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(train_X, train_y)
lr_preds = lr.predict(val_X)
#print(mean_absolute_error(val_y, lr_preds))
print(accuracy_score(val_y, lr_preds))

0.5661157024793388


In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(),
                     LogisticRegression())

pipe.fit(train_X, train_y)
p_preds = pipe.predict(val_X)

print(accuracy_score(val_y, p_preds))

0.5702479338842975
